In [1]:
import pandas as pd
import requests

# Считаем конверсию регистрации

## Загрузка данных

In [19]:
r = requests.get('https://data-charts-api.hexlet.app/visits?begin=2023-03-01&end=2023-09-01')

In [21]:
r.status_code

200

In [24]:
visits = pd.DataFrame(r.json())

In [30]:
visits.shape

(263459, 4)

In [32]:
visits.describe()

,datetime,platform,user_agent,visit_id
count,263459,263459,263459,263459
unique,261204,4,32,146085
top,"Thu, 06 Jul 2023 10:45:50 GMT",web,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109...,64167edd-323a-4ab0-be9b-acd237a1ac30
freq,3,236301,13623,4


In [26]:
r = requests.get('https://data-charts-api.hexlet.app/registrations?begin=2023-03-01&end=2023-09-01')

In [27]:
r.status_code

200

In [28]:
registrations = pd.DataFrame(r.json())

In [29]:
registrations.head()

,datetime,email,platform,registration_type,user_id
0,"Wed, 01 Mar 2023 07:40:13 GMT",ebyrd@example.org,web,google,2e0f6bb8-b029-4f45-a786-2b53990d37f1
1,"Wed, 01 Mar 2023 13:14:00 GMT",knightgerald@example.org,web,email,f007f97c-9d8b-48b5-af08-119bb8f6d9b6
2,"Wed, 01 Mar 2023 03:05:50 GMT",cherylthompson@example.com,web,apple,24ff46ae-32b3-4a74-8f27-7cf0b8f32f15
3,"Wed, 01 Mar 2023 00:04:47 GMT",halldavid@example.org,web,email,3e9914e1-5d73-4c23-b25d-b59a3aeb2b60
4,"Wed, 01 Mar 2023 18:31:52 GMT",denise86@example.net,web,google,27f875fc-f8ce-4aeb-8722-0ecb283d0760


In [33]:
registrations.shape

(21836, 5)

In [34]:
registrations.describe()

,datetime,email,platform,registration_type,user_id
count,21836,21836,21836,21836,21836
unique,21821,20868,3,4,21836
top,"Fri, 23 Jun 2023 13:10:45 GMT",ujones@example.com,android,email,2e0f6bb8-b029-4f45-a786-2b53990d37f1
freq,2,6,10582,8996,1


## Предобработка данных

In [37]:
# провека на пропуски
visits.isna().sum()

datetime      0
platform      0
user_agent    0
visit_id      0
dtype: int64

In [44]:
# удаление дубликотов
visits = visits.drop_duplicates(subset='visit_id', keep='last')

In [69]:
# удаление посещения ботами
bot_visits_mask = visits.user_agent.str.contains(pat='bot', case=False)
visits_clean = visits[~bot_visits_mask]

In [79]:
visits_clean.shape

(138703, 4)

In [81]:
registrations.isna().sum()

datetime             0
email                0
platform             0
registration_type    0
user_id              0
dtype: int64

In [82]:
registrations.duplicated(subset='user_id').sum()

0

In [188]:
visits_clean.head()

,datetime,platform,user_agent,visit_id
0,"Wed, 01 Mar 2023 10:36:22 GMT",web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,1de9ea66-70d3-4a1f-8735-df5ef7697fb9
1,"Wed, 01 Mar 2023 06:25:00 GMT",web,Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7...,f149f542-e935-4870-9734-6b4501eaf614
2,"Wed, 01 Mar 2023 10:26:13 GMT",web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,08f0ebd4-950c-4dd9-8e97-b5bdf073eed1
3,"Wed, 01 Mar 2023 12:33:06 GMT",web,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,19322fed-157c-49c6-b16e-2d5cabeb9592
4,"Wed, 01 Mar 2023 01:38:35 GMT",web,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,04762a22-3c9f-40c9-9ac9-6628c4381836


In [191]:
#приведение даты к формату дейттайм
visits_clean['datetime'] = pd.to_datetime(visits_clean['datetime'])

/var/folders/w6/n_fcyv095mx312wbmf2httjc0000gn/T/ipykernel_3509/3799235007.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visits_clean['datetime'] = pd.to_datetime(visits_clean['datetime'])


In [194]:
#выделние даты из даты+время 
visits_clean['date'] = pd.to_datetime(visits_clean.datetime.dt.date)

/var/folders/w6/n_fcyv095mx312wbmf2httjc0000gn/T/ipykernel_3509/3737517942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  visits_clean['date'] = pd.to_datetime(visits_clean.datetime.dt.date)


In [127]:
registrations['datetime'] = pd.to_datetime(registrations['datetime'])

In [132]:
registrations['date'] = pd.to_datetime(registrations.datetime.dt.date)

## Агрегация данных по дате и платформе

In [196]:
visits_count = visits_clean \
    .groupby(['date', 'platform'], as_index=False) \
    .agg(visits=('visit_id', 'count'))

In [136]:
registrations_count = registrations \
    .groupby(['date', 'platform'], as_index=False) \
    .agg(registrations=('user_id', 'count'))

## Объединение датафреймов по дате и платформе и подсчет конверсии

In [203]:
conversion = visits_count \
    .merge(registrations_count, how='left', on=['date', 'platform']) \
    .rename(columns={'date': 'date_group'})

In [206]:
conversion['conversion'] = conversion.registrations / conversion.visits * 100

In [207]:
conversion = conversion.sort_values('date_group')

In [209]:
conversion.head()

,date_group,platform,visits,registrations,conversion
0,2023-03-01,android,75,61,81.333333
1,2023-03-01,ios,22,18,81.818182
2,2023-03-01,web,787,8,1.016518
3,2023-03-02,android,67,59,88.059701
4,2023-03-02,ios,31,24,77.419355


## Запись полученных данных

In [211]:
conversion.to_json('./conversion.json')

# Добавляем рекламные кампании

In [237]:
ads = pd.read_csv('ads.csv')

In [229]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          159 non-null    datetime64[ns]
 1   utm_source    159 non-null    object        
 2   utm_medium    159 non-null    object        
 3   utm_campaign  159 non-null    object        
 4   cost          159 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 6.3+ KB


In [230]:
ads.isna().sum()

date            0
utm_source      0
utm_medium      0
utm_campaign    0
cost            0
dtype: int64

In [231]:
ads.duplicated().sum()

0

In [243]:
ads['date_group'] = pd.to_datetime(ads.date).dt.floor('D')

In [250]:
ads_short = ads[['date_group', 'cost', 'utm_campaign']]

In [259]:
conversion_short = conversion \
    .groupby('date_group', as_index=False) \
    .agg({'visits': 'sum', 'registrations': 'sum'})

In [260]:
conversion_short.head()

,date_group,visits,registrations
0,2023-03-01,884,87
1,2023-03-02,1050,106
2,2023-03-03,1147,107
3,2023-03-04,1232,159
4,2023-03-05,1145,115


In [262]:
ads_conversion = conversion_short.merge(ads_short, how='left', on='date_group')

In [269]:
ads_conversion = ads_conversion.fillna({'cost': 0, 'utm_campaign': 'none'})

In [270]:
ads_conversion.to_json('./ads.json')